*Some trivial examples to demonstrate core principles of the language, displaying other syntax along the way.*

# Functions

Let's define a function:

In [1]:
function fact(n)
    if n == 1
        n
    else
        n * fact(n-1)
    end
end

fact (generic function with 1 method)

In [2]:
fact(15)

1307674368000

In [3]:
using Test  # standard library package
@test fact(15) == Base.factorial(15)

Test Passed

Julia's JIT works like a static compiler, and generates high-quality machine code:

In [4]:
@code_native fact(15)

	.text
	pushq	%rbx
	movq	%rdi, %rbx
	cmpq	$1, %rbx
	jne	L17
	movl	$1, %eax
	popq	%rbx
	retq
L17:
	leaq	-1(%rbx), %rdi
	movabsq	$fact, %rax
	callq	*%rax
	imulq	%rbx, %rax
	popq	%rbx
	retq
	nopw	(%rax,%rax)


In [5]:
fact(30)

-8764578968847253504

In [6]:
typeof(ans)

Int64

Being fast means compromises: no `BigInt`s *by default*:

In [7]:
fact(big(30))

265252859812191058636308480000000

In [8]:
fact(30.0)

2.6525285981219103e32

In [9]:
@code_native fact(30.0)

	.text
	pushq	%rax
	movabsq	$140215587068080, %rax  # imm = 0x7F867C4258B0
	vucomisd	(%rax), %xmm0
	jne	L19
	jnp	L57
L19:
	movabsq	$140215587068088, %rax  # imm = 0x7F867C4258B8
	vmovsd	%xmm0, (%rsp)
	vaddsd	(%rax), %xmm0, %xmm0
	movabsq	$fact, %rax
	callq	*%rax
	vmulsd	(%rsp), %xmm0, %xmm0
	popq	%rax
	retq
L57:
	vmovsd	(%rax), %xmm0           # xmm0 = mem[0],zero
	popq	%rax
	retq
	nop


## Why start with functions?

Functions are important!
* compiled by the JIT
* known local state

In [10]:
using BenchmarkTools  # external package, more on this later

In [11]:
a = 1
b = 2
@btime sin(a)+b;

  38.947 ns (2 allocations: 32 bytes)


In [12]:
myfunction(a, b) = sin(a)+b
@btime myfunction(1,2);

  11.595 ns (0 allocations: 0 bytes)


# Macro's

* functions: work with values
* macros: work with expressions

Very powerful! Only describe shortly to explain their use.

In [13]:
a = [1, 2, 3]

3-element Array{Int64,1}:
 1
 2
 3

In [14]:
acc = 0
for i in a
    @show acc += i
end
@test acc == sum(a)

acc += i = 1
acc += i = 3
acc += i = 6


┌ Warning: Deprecated syntax `implicit assignment to global variable `acc``.
│ Use `global acc` instead.
└ @ nothing none:0


Test Passed

In [15]:
?@show

```
@show
```

Show an expression and result, returning the result.


In [16]:
@which 1+1

+(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:53

In [17]:
@code_llvm fact(15)


define i64 @julia_fact_37057.1(i64) {
top:
  %1 = icmp eq i64 %0, 1
  br i1 %1, label %L3, label %L4

L3:                                               ; preds = %top
  ret i64 1

L4:                                               ; preds = %top
  %2 = add i64 %0, -1
  %3 = call i64 @julia_fact_37057(i64 %2)
  %4 = mul i64 %3, %0
  ret i64 %4
}


Great for:
* creating DSLs
* adding/prototyping syntax

# Types

In [18]:
typeof(42)

Int64

In [19]:
Int === Int64

true

Flat hierarchy of many abstract and one concrete type:

In [20]:
hierarchy = [Int]
T = last(hierarchy)
while T !== supertype(T)
    push!(hierarchy, supertype(T))
    T = last(hierarchy)
end
hierarchy

6-element Array{DataType,1}:
 Int64  
 Signed 
 Integer
 Real   
 Number 
 Any    

In [21]:
map(T -> (isconcretetype(T), isabstracttype(T), T), hierarchy)

6-element Array{Tuple{Bool,Bool,DataType},1}:
 (true, false, Int64)  
 (false, true, Signed) 
 (false, true, Integer)
 (false, true, Real)   
 (false, true, Number) 
 (false, true, Any)    

Defining our own types is easy:

In [22]:
struct SpecialNumber <: Number
    x::Int
end

In [23]:
x = SpecialNumber(42)

SpecialNumber(42)

Types can be parametric, eg. `Array{T,N}` which is parametic in:
* element type `T`
* dimensions `N`

In [24]:
Vector

Array{T,1} where T

In [25]:
[SpecialNumber(i) for i in 1:10]

10-element Array{SpecialNumber,1}:
  SpecialNumber(1)
  SpecialNumber(2)
  SpecialNumber(3)
  SpecialNumber(4)
  SpecialNumber(5)
  SpecialNumber(6)
  SpecialNumber(7)
  SpecialNumber(8)
  SpecialNumber(9)
 SpecialNumber(10)

In [26]:
special_mat = SpecialNumber.(rand(1:100, 10, 10))

10×10 Array{SpecialNumber,2}:
 SpecialNumber(55)   SpecialNumber(69)  …   SpecialNumber(27)
 SpecialNumber(42)   SpecialNumber(98)      SpecialNumber(98)
 SpecialNumber(61)   SpecialNumber(42)       SpecialNumber(9)
 SpecialNumber(13)   SpecialNumber(32)      SpecialNumber(14)
 SpecialNumber(87)   SpecialNumber(83)      SpecialNumber(92)
 SpecialNumber(59)   SpecialNumber(90)  …   SpecialNumber(75)
  SpecialNumber(8)  SpecialNumber(100)      SpecialNumber(32)
 SpecialNumber(76)   SpecialNumber(66)      SpecialNumber(77)
 SpecialNumber(18)   SpecialNumber(71)     SpecialNumber(100)
 SpecialNumber(39)   SpecialNumber(33)      SpecialNumber(56)

# Multiple dispatch

Single-dispatch languages: `SpecialNumber::plus(SpecialNumber other)`

Multiple dispatch: much more natural way of expressing behavior.

In [27]:
function Base.:+(a::SpecialNumber, b::SpecialNumber)
    SpecialNumber(a.x + b.x)
end

In [28]:
x + x

SpecialNumber(84)

In [29]:
sum(SpecialNumber[x, x, x])

SpecialNumber(126)

Defining multiplication gets us matrix multiplication:

In [30]:
Base.:*(a::SpecialNumber, b::SpecialNumber) = SpecialNumber(a.x * b.x)
special_mat * special_mat

10×10 Array{SpecialNumber,2}:
 SpecialNumber(21141)  SpecialNumber(31268)  …  SpecialNumber(26442)
 SpecialNumber(28699)  SpecialNumber(40700)     SpecialNumber(37391)
 SpecialNumber(28640)  SpecialNumber(40429)     SpecialNumber(38005)
 SpecialNumber(28686)  SpecialNumber(44050)     SpecialNumber(36293)
 SpecialNumber(25935)  SpecialNumber(40357)     SpecialNumber(33827)
 SpecialNumber(19513)  SpecialNumber(31885)  …  SpecialNumber(29510)
 SpecialNumber(21441)  SpecialNumber(39744)     SpecialNumber(34568)
 SpecialNumber(24344)  SpecialNumber(33290)     SpecialNumber(31275)
 SpecialNumber(20494)  SpecialNumber(33873)     SpecialNumber(27137)
 SpecialNumber(16194)  SpecialNumber(31192)     SpecialNumber(24965)

What if we want to work with other numbers?

In [31]:
Base.:+(a::SpecialNumber, b::Number) = SpecialNumber(a.x + b)
Base.:+(a::Number, b::SpecialNumber) = SpecialNumber(a + b.x)

In [32]:
x + 1

SpecialNumber(43)

In [33]:
1 + x

SpecialNumber(43)

This last definition would have been hard in a single-dispatch language!

For example, Python: `__add__` and `__radd__`; doesn't work generally.